# Исследование надежности заемщиков.

Заказчик — кредитный отдел банка. 
В ходе работы нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок.

Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

**Цель исследования**
Входные данные от банка — статистика о платёжеспособности клиентов. На их основе мы ответим на следующие вопросы:
1. Есть ли зависимость между количеством детей и возвратом кредита в срок?
2. Есть ли зависимость между семейным положением и возвратом кредита в срок?
3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
4. Как разные цели кредита влияют на его возврат в срок?

**Ход исследования**

Данные о поведении пользователей мы получаем из файла `/datasets/data.csv`. О качестве данных ничего не известно. Поэтому перед проверкой гипотез понадобится обзор данных. 

Мы проверим данные на ошибки и оценим их влияние на исследование. Затем, на этапе предобработки мы поищем возможность исправить самые критичные ошибки данных.
 
Таким образом, исследование пройдёт в три этапа:
 1. [Обзор данных.](#explore) 
 2. [Предобработка данных.](#preprocessing)
 3. [Ответы на вопросы и выводы.](#conclusion)

## Шаг 1. Обзор данных <a id='explore'></a>
Откроем файл с данными и посмотрим на общую информацию и состав данных.

In [1]:
# импорт библиотек
import pandas as pd
import os
import numpy as np

In [2]:
# проверка на путь
if os.path.exists('/datasets/data.csv'):
    path = '/datasets/'
else:
    path = './'
    
# загружаем датасет в df
df = pd.read_csv(path+'data.csv')

# проверка загрузки данных и отображение первых 10 строк
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [3]:
# также посмотрим на случайную выборку данных, т.к. в начале и в конце таблицы 
# данные могут сильно отличаться от всего набора
# получение случайных 10 строк таблицы df
df.sample(10, random_state=42)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
423,0,-191.167760,56,среднее,1,женат / замужем,0,M,сотрудник,0,138653.748793,автомобили
3522,0,-2319.817259,58,среднее,1,Не женат / не замужем,4,F,сотрудник,0,123152.627177,покупка жилья
8760,1,-2990.578297,34,высшее,0,гражданский брак,1,M,компаньон,0,232380.737167,свадьба
20695,0,-926.209452,28,среднее,1,Не женат / не замужем,4,F,сотрудник,0,210617.086601,получение высшего образования
4351,0,-2524.302106,42,СРЕДНЕЕ,1,женат / замужем,0,F,сотрудник,0,78487.540219,сделка с автомобилем
3002,0,NaN,28,высшее,0,женат / замужем,0,M,компаньон,0,NaN,покупка жилой недвижимости
17306,0,-5000.830630,0,среднее,1,гражданский брак,1,F,компаньон,0,83019.106740,свадьба
2830,0,-574.995923,40,среднее,1,женат / замужем,0,F,сотрудник,0,161158.806308,автомобили
14822,2,NaN,33,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,заняться образованием
10347,1,-713.140231,48,среднее,1,женат / замужем,0,M,компаньон,0,177474.885995,ремонт жилью


In [4]:
# получение общей информации о данных в таблице df
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Итак, в таблице 12 столбцов. 

Согласно документации к данным:

* `children` — количество детей в семье
* `days_employed` — общий трудовой стаж в днях 
* `dob_years` — возраст клиента в годах
* `education` — уровень образования клиента
* `education_id` — идентификатор уровня образования
* `family_status` — семейное положение
* `family_status_id` — идентификатор семейного положения
* `gender` — пол клиента
* `income_type` — тип занятости
* `debt` — имел ли задолженность по возврату кредитов
* `total_income` — ежемесячный доход
* `purpose` — цель получения кредита

Всего 21525 строк, но кол-во значений в столбцах различается:
* `days_employed`     19351
* `total_income`      19351

Значит, в этих столбцах пропущены значения, обработаем эти пропуски.

Также из первых и случайных 10-ти строк видно, что есть ошибки в данных столбцов:
* `days_employed` - отрицательные значения
* `education` - разный регистр

Скорее всего, это не все ошибки. Внимательно изучим каждый слолбец и проверим, можем ли мы исправить ошибки.


Выводы

В каждой строке таблицы — данные о заемщике. Большая часть колонок описывает заемщика: возраст, стаж, образование, кол-во детей, доход, семейное положение. Две колонки содержат информацию о кредите: задолженность и цель. Еще две колонки являются инетификаторами других колонок: семейного положения и образования.

Предварительно можно утверждать, что, данных достаточно для ответа на поставленные вопросы. Но встречаются пропуски и ошибки в данных, а две колонки дублируют другие по сути значений.

Чтобы данные были более точные, нужно устранить проблемы в данных.

## Шаг 2. Предобработка данных <a id='preprocessing'></a>
### Шаг 2.1 Заполнение пропусков

Посмотрим, сколько пропусков есть в таблице и в каких колонках.

In [5]:
# подсчет пропусков и визуализация процентного соотношения пропусков
display(pd.DataFrame(round((df.isna().mean()*100),2), columns=['NaNs, %']).style.format(
    '{:.2f}').background_gradient('bwr'))
df.isna().sum()

,"NaNs, %"
children,0.00
days_employed,10.10
dob_years,0.00
education,0.00
education_id,0.00
family_status,0.00
family_status_id,0.00
gender,0.00
income_type,0.00
debt,0.00


children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

В двух столбцах есть пропуски:
* `days_employed` - 2174
* `total_income` - 2174

2174 это 10% от всего набора данных 21525. Достаточно много пропусков.

Данные `days_employed` - трудовой стаж в днях не участвует в исследовании. А `total_income` - ежемесячный доход влияет на один из поставленных вопросов.

На практике было бы правильно установить причину пропусков и восстановить данные. Такой возможности нет в учебном проекте. Придётся:

- заполнить и эти пропуски подходящими значениями,
- оценить, насколько они повредят расчётам.

Из общих данных мы видим, что обе колонки в формате `float64` и содержат количественные переменные. Для того, чтобы заполнить пропуски, посмотрим на крайние значения в этих колонках.

In [6]:
# выводим уникальные значения days_values
df.sort_values(by='days_employed')['days_employed'].unique()

array([-18388.94990057, -17615.56326563, -16593.47281726, ...,
       401715.81174889, 401755.40047533,             nan])

In [7]:
# выводим уникальные значения total_income
df.sort_values(by='total_income')['total_income'].unique()

array([  20667.26379327,   21205.28056622,   21367.64835649, ...,
       2200852.2102589 , 2265604.02872274,              nan])

Т.к. в указанных столбцах количественные перменные, их можно заполнить средним или медианным значением. В столбце `days_employed` видим разброс значений от отрицательных, до положительных чисел, заполнение пропусков приведет к искажению данных. А в столбце `total_income` значения положительные и данные от 20-ти тысяч рублей до 2-х миллионов выглядят реальными. Но разброс очень большой, поэтому заполнение медианой меньше искозит данные, чем заполнение средним значением.

Можно предположить, что пропуски в колонке с ежемесячным доходом возникли из-за того, что человек не работает. Проверим какие типы занятости у заемщиков с пропущенными значениями в в ежемесячном доходе.

In [8]:
# фильтруем значения с NaN в колонке total_income и выведим уникальные значения для колонки income_type
df[df['total_income'].isna()]['income_type'].unique()

array(['пенсионер', 'госслужащий', 'компаньон', 'сотрудник',
       'предприниматель'], dtype=object)

Видим, что типы занятости не предполагают нулевого дохода. Значит это техническая ошибка или пропущенные данные при заполнении. Заполним пропуски в столбце `total_income` медианным значением. Чтобы данные были ближе к истинным, вычислим медиану для каждой группы по типу занятости и уровню образования.

Колонка `income_type` участвует в исследовании. Поэтому для оценки искажения результатов изменением данных, отметим в каких строчках были внесены изменения в новой колонке `flag_change_income` 

In [9]:
# отмечаем 1 в flag_change_income строки с пропущенными данными в total_income
df.loc[df['total_income'].isna(),'flag_change_income']=1

# смотрим первые 10 строчек с пропущенными данными в total_income
df[df['total_income'].isna()].head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,flag_change_income
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу,1.0
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование,1.0
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости,1.0
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем,1.0
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу,1.0
65,0,NaN,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,NaN,операции с коммерческой недвижимостью,1.0
67,0,NaN,52,высшее,0,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья для семьи,1.0
72,1,NaN,32,высшее,0,женат / замужем,0,M,госслужащий,0,NaN,операции с коммерческой недвижимостью,1.0
82,2,NaN,50,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,жилье,1.0
83,0,NaN,52,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,жилье,1.0


In [10]:
# заполнение пропусков в колонке total_income медианными значениями с группировкой по типу занятости и уровню образования.
df['total_income'].fillna(df.groupby(['income_type','education_id'])['total_income'].transform('median'),inplace = True)

# отображаем строчки, в которых было выполнено заполнение
display(df[df['flag_change_income']==1])

# подсчёт пропусков
df.isna().sum()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,flag_change_income
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,114842.854099,сыграть свадьбу,1.0
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,136652.970357,образование,1.0
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,114842.854099,строительство жилой недвижимости,1.0
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,136652.970357,сделка с подержанным автомобилем,1.0
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,114842.854099,сыграть свадьбу,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,159070.690289,сделка с автомобилем,1.0
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,136555.108821,свадьба,1.0
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,201785.400018,строительство недвижимости,1.0
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,136555.108821,строительство жилой недвижимости,1.0


children                  0
days_employed          2174
dob_years                 0
education                 0
education_id              0
family_status             0
family_status_id          0
gender                    0
income_type               0
debt                      0
total_income              0
purpose                   0
flag_change_income    19351
dtype: int64

2174 пропусков заполнили медианой.

Пропусков в колонке `total_income` нет. А для заполнения пропусков в колонке `days_employed` сначала надо разобраться с аномалиями.

### Шаг 2.2 Проверка данных на аномалии и исправления.
Мы знаем, что в колонке `days_employed` есть пропуски, и отрицательные значения. Для ответов на поставленные проектом вопросы, трудовой стаж не требуется и можно было бы игнорировать эту колонку полностью. Но для отработки учебного процесса, посмотрим, что можно сделать с этими данными, на случай, если потребуется их дальнейшее использование.

Из данных выше видно, минимальное значение стажа -18388 дней (50 лет), а максимальное положительное значение в трудовом стаже 401755дней. Это 401755/365 = 1100 лет. Точно аномалия, учитывая, что средний рабочий стаж при выходе на пенсию в РФ 35 лет = 12775 дней. Посмотрим на положительные значения `days_employed`.

In [11]:
# фильтруем и выводим положительные значения в колонке days_employed
df[df['days_employed']>0].sort_values(by='days_employed')['days_employed'].unique()

array([328728.72060452, 328734.92399633, 328771.3413868 , ...,
       401675.09343386, 401715.81174889, 401755.40047533])

Минимальное положительное значение больше 900 лет. Следовательно все положительные значения или аномальны или записаны не в формате дней. Предположим, что значения записаны в часах, тогда минимальное положительное значение 37 лет, а максимальное 45 лет. Похоже на правду, но надо сравнить еще с возрастом этих заемщиков.

Посмотрим сколько таких заемщиков с аномально-большим стажем. Если процент не большой, то ими можно будет пренебречь.

In [12]:
# посчитаем положительные значения в колонке days_employed
df[df['days_employed']>0]['days_employed'].count()

3445

3445 заемщика - 16% от общего числа. Достаточно большая доля для того, чтобы удалять эти данные.

Посмотрим на тип занятости этих заемщиков.

In [13]:
# Подсчёт количества заемщиков по типу занятости с положительным стажем работы
df[df['days_employed']>0].groupby(by='income_type')['income_type'].count()

income_type
безработный       2
пенсионер      3443
Name: income_type, dtype: int64

Всего 2 безработных, которыми можно пренебречь, а остальные пенсионеры, для которых стаж 37-45 лет вполне допустим. Посмотрим на возраст этих заемщиков.

In [14]:
# Подсчёт количества заемщиков по возрасту с положительным стажем работы
df[df['days_employed']>0].groupby(by='dob_years')['dob_years'].count()

dob_years
0      17
22      1
26      2
27      3
28      1
31      1
32      3
33      2
34      3
35      1
36      3
37      5
38      8
39      4
40      7
41      6
42      9
43      9
44     10
45     11
46     13
47     13
48     20
49     30
50     61
51     73
52     95
53    105
54    145
55    162
56    184
57    212
58    208
59    254
60    243
61    214
62    235
63    192
64    179
65    136
66    139
67    132
68     80
69     74
70     54
71     48
72     28
73      6
74      4
Name: dob_years, dtype: int64

В возрасте видно, что тоже есть аномалии. Посмотрим на уникильные данные в столбце `dob_years`

In [15]:
# сортируем и выводим уникальные значения в колонке dob_years       
df.sort_values(by='dob_years')['dob_years'].unique()

array([ 0, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73, 74, 75], dtype=int64)

Верхнее значение выглядит вполне правдоподобно. А заемщиков младше 18 лет быть не должно. 

Для ответов на поставленные проектом вопросы, возраст не требуется и можно было бы игнорировать эту колонку полностью. Но для отработки учебного процесса, посмотрим, что можно сделать с этими данными, на случай, если потребуется их дальнейшее использование.

Посмотрим сколько заемщиков с возрастом 0.

In [16]:
# фильтруем и считаем значения 0 в колонке dob_years
df[df['dob_years']==0]['dob_years'].count()


101

Меньше, чем 0,5% от общего числа. Эта ошибка может быть технической или случайной. Такой небольшой процент с аномалиями можно игнорировать, удалить или заменить на подходящие данные. Т.к. дальше с возрастом будет связано заполнение данных по стажу, заполним нулевые значения медианным.

--------------------------------------

! Важное отступление !

В рабочем проекте на этом этапе я бы остановилась с обработкой данных из столбца `days_employed` и попросила бы их исправить/восстановить, если на их основе проводилось бы исследование. Т.к. такое большое кол-во неточностей при исправлении большею частью догадками без доказательной базы, могут привести к большому искажению. Но, с учетом того, что проект учебный и данные из колонок `days_employed` и `dob_years` в исследовании не участвуют и не исказят его, я попробую исправить ошибки максимально приближенными к реальности данными. 

Т.к. эти исправления могут повлиять на дальнейшую работу в части удаления явных дубликатов, посчитаем такие дубликаты сейчас, установим их долю и сравним с долей после заполнения полей `days_employed` и `dob_years`

--------------------------------------



In [17]:
# считаем кол-во явных дубликатов
df.duplicated().sum()

54

54 дубликата - 0,25% очень маленькая доля от общего числа. Удалим эти строчки.

In [18]:
df=df.drop_duplicates()
# проверяем кол-во явных дубликатов
df.duplicated().sum()

0

Явные дубликаты удалены.

Приступаем к удалению аномалий в колонках `days_employed` и `dob_years`

In [19]:
# заполнение 0 в колонке dob_years медианными значениями с группировкой семейному положению и типу занятости.
df.loc[df['dob_years']==0, 'dob_years']=df.groupby(['family_status_id','income_type'])['dob_years'].transform('median')

# сортируем и выводим уникальные значения в колонке dob_years для проверки       
df.sort_values(by='dob_years')['dob_years'].unique()

array([19., 20., 21., 22., 23., 24., 25., 26., 27., 28., 29., 30., 31.,
       32., 33., 34., 35., 36., 37., 38., 39., 40., 41., 42., 43., 44.,
       45., 46., 47., 48., 49., 50., 51., 52., 53., 54., 55., 56., 57.,
       58., 59., 60., 61., 62., 63., 64., 65., 66., 67., 68., 69., 70.,
       71., 72., 73., 74., 75.])

Возраст отображается без нулей, но формат стал вещественным числом. Для ближайшей работы формат не помешает, позже его изменим.

Возвращаемся к стажу работы. Допустим, что положительный стаж указан в часах, а не днях. В этом случае разброс 37-45 выглядит правдиво, а разница между стажем и возрастом более, чем 18 лет прибавит веса в этом утверждении.

Создадим колонку `years_employed` в `df` и поместим туда пересчитанные положительные значения из `days_employed` в годы.

In [20]:
def employed_minutes_years(row):
    """
    Функция возвращает стаж работы в годах
    """
    days_employed=row['days_employed']
    
    if days_employed>0:
        return days_employed/365/24

# добавляем колонку years_employed в df
df['years_employed']=df.apply(employed_minutes_years,axis=1)

# смотрим, какие значения получились в years_employed
df.groupby(by='years_employed')['years_employed'].count()

years_employed
37.526110    1
37.526818    1
37.530975    1
37.533759    1
37.537368    1
            ..
45.852038    1
45.853250    1
45.853321    1
45.857969    1
45.862489    1
Name: years_employed, Length: 3445, dtype: int64

37-45 ожидаемые результаты. Теперь можно посмотреть на разницу в возрате и стаже.

Создадим колонку `difference` в `df` и поместим туда разницу `dob_years` и `days_employed`для положительного стажа.

In [21]:
def difference_employed_dob(row):
    """
    Функция возвращает разницу между возрастом и стажем работы для положительного стажа
    """
    days_employed=row['days_employed']
    dob_years=row['dob_years']
    years_employed=row['years_employed']
    
    if days_employed>0:
        return dob_years-years_employed

# добавляем колонку difference в df
df['difference']=df.apply(difference_employed_dob,axis=1)

# смотрим, какие значения получились в difference
df.groupby(by='difference')['difference'].count()

difference
-18.451731    1
-16.321016    1
-16.215098    1
-16.016505    1
-15.592260    1
             ..
 33.742551    1
 34.473890    1
 34.558466    1
 34.659379    1
 35.012771    1
Name: difference, Length: 3445, dtype: int64

Минимальное значение не может превышать 18 лет, а мы видим даже отрицательные значения. Но 17 строк с положительным стажем были изначально с 0 значением в возрасте. Возможно именно они дали искажение после внесения правок с колонку `dob_years`.

Посмотрим, сколько значений меньше 18 в колонке `difference`.

In [22]:
# фильтруем и считаем значения 0 в колонке difference
df[df['difference']<18]['difference'].count()

1717

1717 Даже не близко к 17 замененным.

Трудовой договор разрешается заключать с подростками, достигшими возраста 16 лет (ст. 63 ТК РФ). Для выполнения легкого труда, не причиняющего вред здоровью, можно принять 15-летнего работника, если он получил общее образование либо продолжает обучение по основной программе общего образования по вечерней или заочной форме. Допускается наем 15-летнего и в случае, когда он оставил учебу.

Опустим минимальную планку до 15 лет.

In [23]:
# фильтруем и считаем значения 0 в колонке difference
df[df['difference']<15]['difference'].count()

1061

1061 это треть от всех заемщиков с положительным стажем. Значит, положительные данные в колонке `days_employed` вряд ли были записаны в формате часы.

Посмотрим на отрицательные значения.

In [24]:
# фильтруем и считаем уникальные отрицательные значения days_values 
df[df['days_employed']<=0]['days_employed'].count()

15906

In [25]:
# фильтруем, сортируем и выводим уникальные отрицательные значения days_values 
df[df['days_employed']<=0].sort_values(by='days_employed')['days_employed'].unique()

array([-18388.94990057, -17615.56326563, -16593.47281726, ...,
          -30.19533716,    -24.24069479,    -24.14163324])

Отрицательных значений довольно много 15906. 74% от общего числа. При этом разброс абсолютных отрицательных значений стажа выглядит правдоподобнее (от 24 дней до 50 лет). Нулевых значений нет.

Допустим, что отрицательные значения стажа были по ошибке записаны со знаком "-", тогда разница между возрастом и абсолютным отрицательным стажем будет не менее 15-18 лет.

In [26]:
# удаляем столбцы years_employed и difference из df
df=df.drop(['years_employed', 'difference'], axis=1)

Создадим колонку `difference` в `df` и поместим туда разницу `dob_years` и `days_employed`для отрицательного стажа.

In [27]:
def difference_employed_dob(row):
    """
    Функция возвращает разницу между возрастом и стажем работы
    """
    days_employed=row['days_employed']
    dob_years=row['dob_years']
    
    if days_employed<0:
        return dob_years-abs(days_employed)/365

# добавляем колонку difference в df
df['difference']=df.apply(difference_employed_dob,axis=1)

# смотрим, какие значения получились в difference
df.groupby(by='difference')['difference'].count()

difference
10.619315    1
10.714382    1
10.806890    1
10.857530    1
11.736114    1
            ..
68.666679    1
68.979765    1
68.997604    1
69.261281    1
70.400083    1
Name: difference, Length: 15906, dtype: int64

Минимальное значение 10. Не правдоподобно, но нельзя забывать, что 84 строки с 0 возрастом были заполнены медианой.
Максимальный возраст в данных 75 лет. Максимальные значение разницы - 70 не превышает 75.
Посмотрим, сколько значений меньше 18 в колонке `difference`.

In [28]:
# фильтруем и считаем значения в колонке difference
df[df['difference']<18]['difference'].count()

409

409 много. Допустим, что 300 из них начали стаж в период 15-18 лет.

In [29]:
# фильтруем и считаем значения в колонке difference
df[df['difference']<15]['difference'].count()

46

46 меньше, чем 84. Значит можно списать эту разницу 10-15 лет на заполнение 0 возраста.

Можно допустить, что отрицательные значения в абсолютном пересчете будут верными. И, из всех возможных вариантов заполнения (кроме восстановления данных у заказчика) меньше всего искажений принесет следующий алгоритм:
- удалить из столбца `difference` значения меньше 15
- заполнить все пропуски `difference` медианой по этому столбцу с группировкой по возрасту
- удалить из столбца `days_employed` все положительные значения
- заполнисть все пропуски столбца `days_employed` разницей `dob_years` и `difference`
- заменить отрицательные значения в столбце `days_employed` на их абсолютные значения

In [30]:
# удаляем из столбца difference значения меньше 15
df.loc[df['difference']<15,'difference']=df.drop(columns='difference')

# смотрим, какие значения получились в difference
df.groupby(by='difference')['difference'].count()

difference
15.016952    1
15.024018    1
15.079178    1
15.079578    1
15.086162    1
            ..
68.666679    1
68.979765    1
68.997604    1
69.261281    1
70.400083    1
Name: difference, Length: 15860, dtype: int64

In [31]:
# подсчет пропусков
df.isna().sum()

children                  0
days_employed          2120
dob_years                 0
education                 0
education_id              0
family_status             0
family_status_id          0
gender                    0
income_type               0
debt                      0
total_income              0
purpose                   0
flag_change_income    19351
difference             5611
dtype: int64

In [32]:
# заполняем пропуски difference медианой по этому столбцу с группировкой по возрасту
df['difference'].fillna(df.groupby(['dob_years'])['difference'].transform('median'),inplace = True)

# подсчет пропусков
df.isna().sum()

children                  0
days_employed          2120
dob_years                 0
education                 0
education_id              0
family_status             0
family_status_id          0
gender                    0
income_type               0
debt                      0
total_income              0
purpose                   0
flag_change_income    19351
difference                0
dtype: int64

В столбцe `difference` пропусков нет.

In [33]:
# удаляем из столбца days_employed все положительные значения
df.loc[df['days_employed']>0,'days_employed']=df.drop(columns='days_employed')

# смотрим, какие значения получились в days_employed
df.groupby(by='days_employed')['days_employed'].count()

days_employed
-18388.949901    1
-17615.563266    1
-16593.472817    1
-16264.699501    1
-16119.687737    1
                ..
-34.701045       1
-33.520665       1
-30.195337       1
-24.240695       1
-24.141633       1
Name: days_employed, Length: 15906, dtype: int64

In [34]:
# заполняем все пропуски столбца days_employed разностью dob_years и difference,
# а отрицательные значения в столбце days_employed на заменяем на их абсолютные значения

def change_days_employed(row):
    """
    Функция возвращает:
    - сумму возраста и разницей между возрастом и стажем работы, если значение days_employed пустое
    - абсолютное значение, если значение days_employed отрицательное
    
    """
    days_employed=row['days_employed']
    dob_years=row['dob_years']
    difference=row['difference']
    
    if days_employed<0:
        return abs(days_employed)
    else:
        return dob_years*365-difference*365

# исправляем данные в days_employed в df
df['days_employed']=df.apply(change_days_employed,axis=1)

# смотрим, какие значения получились в days_employed
df.groupby(by='days_employed')['days_employed'].count()


days_employed
24.141633       1
24.240695       1
30.195337       1
33.520665       1
34.701045       1
               ..
16119.687737    1
16264.699501    1
16593.472817    1
17615.563266    1
18388.949901    1
Name: days_employed, Length: 15957, dtype: int64

In [35]:
# подсчет пропусков
df.isna().sum()

children                  0
days_employed             0
dob_years                 0
education                 0
education_id              0
family_status             0
family_status_id          0
gender                    0
income_type               0
debt                      0
total_income              0
purpose                   0
flag_change_income    19351
difference                0
dtype: int64

Пропусков нет, данные в `days_employed` от 24 дней до 50 лет, отрицательных значений нет.
Заполнение пропусков для колонок `days_employed` и `dob_years` в данном случае выполнены только для учебных целей. Скорее всего такое заполнение в рабочем проекте приведет к искажению в анализе этих данных, т.к. изначально данные были очень некорректны.

Столбец `difference` для дальнейшей работы не нужен. Удалим его.

In [36]:
# удаляем колонку difference из df
df=df.drop(['difference'], axis=1)

Теперь посмотрим на колонку `children`

In [37]:
# проверяем уникальные значения в колонке children       
df.sort_values(by='children')['children'].unique()

array([-1,  0,  1,  2,  3,  4,  5, 20], dtype=int64)

0-5 детей в семье вполне может быть. 20, конечно, можно допустить (например, семья берет много приемных детей), но скорее это аномалия, как и значение -1. Посмотрим, сколько аномальных значений в данных.

In [38]:
# Подсчёт количества и процента заемщиков по кол-ву детей
(pd.concat([df['children'].value_counts().sort_index()
            , df['children'].value_counts(normalize = True).map('{:.1%}'.format)]
           , axis=1))

,children,children
-1,47,0.2%
0,14107,65.7%
1,4809,22.4%
2,2052,9.6%
3,330,1.5%
4,41,0.2%
5,9,0.0%
20,76,0.4%


0,2 % со значение -1 (47 шт.)
0,4 % со значением 20 (76 шт.)

Процент аномалий маленький. Данные из этой колонки участвуют в исследовании, поэтому игнорировать нельзя. Судя по значениям -1 и 20 имеет место опечатки при вводе данных. -1 вместо 1 и 20 вместо 2. Т.к. процент не большой, эти строки можно удалить или объединить со значениями 1 и 2.

С 1 ребенком 66% семей, а с 2 - 22%. Объединение не критично исказит данные, зато мы сохраним эти строки для исследования в других столбцах.

In [39]:
# заменим -1 на 1 и 20 на 2 в колонке children
df.loc[df['children']==-1,'children']=1
df.loc[df['children']==20,'children']=2

# Подсчёт количества и процента заемщиков по кол-ву детей
(pd.concat([df['children'].value_counts().sort_index()
            , df['children'].value_counts(normalize = True).map('{:.1%}'.format)]
           , axis=1))

,children,children
0,14107,65.7%
1,4856,22.6%
2,2128,9.9%
3,330,1.5%
4,41,0.2%
5,9,0.0%


В процентном соотношении данные по 1 и 2 детям в семье изменились на десятые доли процентов, что совсем не значительно и критически не исказит результаты исследования.

С колличественными перемеными закончили. Посмотрим на уникальные значения категориальных переменных: `education`, `education_id`, `family_status`, `family_status_id`, `gender`, `income_type`, `debt`, `purpose`

In [40]:
# выводим уникальные значения education
df['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

В колонке с обрзованием много неявных дубликатов, которые мы обработаем в дальнейшем. Аномалий нет.

In [41]:
# выводим уникальные значения education_id
df['education_id'].unique()

array([0, 1, 2, 3, 4], dtype=int64)

В колонке `education_id` значения без аномалий.

In [42]:
# выводим уникальные значения family_status 
df['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

В колонке `family_status` значения без аномалий. Но одно значение отличается по формату - "Не женат / не замужем" с заглавной буквы. Исправим это.

In [43]:
# заменяем "Не женат / не замужем" на "не женат / не замужем" в колонке family_status
df.loc[df['family_status']=='Не женат / не замужем','family_status']='не женат / не замужем'

# выводим уникальные значения family_status для проверки замены
df['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'не женат / не замужем'], dtype=object)

Все уникальные значения в колонке `family_status` в едином стиле, без аномалий.

In [44]:
# выводим уникальные значения family_status_id
df.sort_values(by='family_status_id')['family_status_id'].unique()

array([0, 1, 2, 3, 4], dtype=int64)

В колонке `family_status_id` аномальных значений нет.

In [45]:
# выводим уникальные значения gender
df['gender'].unique()

array(['F', 'M', 'XNA'], dtype=object)

В колонке `gender` 3 значения. На данный момент в РФ официально признается только 2 гендера. Посмотрим, сколько значений приходится на все значения.

In [46]:
# Подсчёт количества заемщиков по полу
df['gender'].value_counts()

F      14189
M       7281
XNA        1
Name: gender, dtype: int64

Всего 1 заемщик с ошибкой в заполнении поля `gender`. Совсем ничтожная доля в данных. Восстановить данные по другим полям не получится. На исследование эта колонка не влияет. Удалим эту строчку.

In [47]:
# удаляем строчку с индексом 10701
df = df[df['gender']!='XNA']

In [48]:
# Подсчёт количества заемщиков по полу
df['gender'].value_counts()

F    14189
M     7281
Name: gender, dtype: int64

Данные в колонке `gender` без аномалий.

In [49]:
# выводим уникальные значения income_type
df['income_type'].unique()

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

In [50]:
# выводим уникальные значения debt
df.sort_values(by='debt')['debt'].unique()

array([0, 1], dtype=int64)

In [51]:
# выводим уникальные значения purpose
df['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

В колонках `income_type`, `debt`, `purpose` уникальные значения без аномалий.

### Шаг 2.3. Изменение типов данных.

Для дальнейшей работы над поставленными вопросами, необходимо посмотреть на типы данных в столбцах:

* `children` имеет целочисленный формат - `int64` и удобна для дальнейшего исследования
* `family_status` имеет строковый формат - `object`, но для дальнейшей работы будем использовать `family_status_id` с целочисленным форматом
* `total_income` данные записаны в фомате вещественных чисел - `float64`, для удобства дальнейшей категоризации переведем в целочисленный формат
* `purpose`- строковый формат `object` оставим без изменений.

In [52]:
# изменяем формат total_income на целочисленный
df['total_income']=df['total_income'].astype('int')

# проверяем формат total_income
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21470 entries, 0 to 21524
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   children            21470 non-null  int64  
 1   days_employed       21470 non-null  float64
 2   dob_years           21470 non-null  float64
 3   education           21470 non-null  object 
 4   education_id        21470 non-null  int64  
 5   family_status       21470 non-null  object 
 6   family_status_id    21470 non-null  int64  
 7   gender              21470 non-null  object 
 8   income_type         21470 non-null  object 
 9   debt                21470 non-null  int64  
 10  total_income        21470 non-null  int32  
 11  purpose             21470 non-null  object 
 12  flag_change_income  2120 non-null   float64
dtypes: float64(3), int32(1), int64(4), object(5)
memory usage: 2.2+ MB


Столбцы `days_employed` и `dob_years` не участвуют в исследовании, но мы вносили в них изменения и приведем их тоже к целочисленному формату.

In [53]:
# изменяем формат days_employed и dob_years на целочисленный
df['days_employed']=df['days_employed'].astype('int')
df['dob_years']=df['dob_years'].astype('int')

# проверяем формат days_employed и dob_years
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21470 entries, 0 to 21524
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   children            21470 non-null  int64  
 1   days_employed       21470 non-null  int32  
 2   dob_years           21470 non-null  int32  
 3   education           21470 non-null  object 
 4   education_id        21470 non-null  int64  
 5   family_status       21470 non-null  object 
 6   family_status_id    21470 non-null  int64  
 7   gender              21470 non-null  object 
 8   income_type         21470 non-null  object 
 9   debt                21470 non-null  int64  
 10  total_income        21470 non-null  int32  
 11  purpose             21470 non-null  object 
 12  flag_change_income  2120 non-null   float64
dtypes: float64(1), int32(3), int64(4), object(5)
memory usage: 2.0+ MB


Формат `total_income`, `days_employed` и `dob_years` изменен на целочисленный. Можно переходить к дубликатам.

### Шаг 2.4. Удаление дубликатов.

In [54]:
# считаем кол-во явных дубликатов
df.duplicated().sum()

0

Явных дубликатов нет. В шаге 2.2 мы уже удалили 54 дубликата. После всех изменений явных дубликатов не прибавилось.

В шаге 2.2 неявные дубликаты были выявлены в колонкe `education`.
Посмотрим на уникальные значения.

In [55]:
# выводим уникальные значения education
df['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

Ошибка скорее всего случайная: не было валидации регистра на вводе данных. Необходимо привести все значения к нижнему регистру.

In [56]:
# приводим значения из колонки education к нижнему регистру
df['education'] = df['education'].str.lower()

# выводим уникальные значения education
df['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

Посмотрим, есть ли явные дубликаты после замены регистра.

In [57]:
# считаем кол-во явных дубликатов
df.duplicated().sum()

17

17 дубликатов. Удаляем.

In [58]:
df=df.drop_duplicates()
# проверяем кол-во явных дубликатов
df.duplicated().sum()

0

Все строки уникальны. Явных и неявных дубликатов в данных нет.

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

На данный момент в датафрейме есть дублирующие друг друга колонки по значению:
* `education` - `education_id`
* `family_status` - `family_status_id`

Текстовые данные в первых занимают много памяти, являются источниками опечаток и продублированы вторыми. Для удобства создадим два новых датафрейма, в которых:
* каждому уникальному значению из `education` соответствует уникальное значение `education_id`
* каждому уникальному значению из `family_status` соответствует уникальное значение `family_status_id`

In [59]:
# создаем словарь education_dict
education_dict=df[['education','education_id']]
education_dict=education_dict.drop_duplicates().reset_index(drop=True)
# отображаем словарь
education_dict

,education,education_id
0,высшее,0
1,среднее,1
2,неоконченное высшее,2
3,начальное,3
4,ученая степень,4


In [60]:
# создаем словарь family_dict
family_dict=df[['family_status','family_status_id']]
family_dict=family_dict.drop_duplicates().reset_index(drop=True)
# отображаем словарь
family_dict

,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,не женат / не замужем,4


Словари `education_dict` и `family_dict` созданы и теперь можно удалить из исходного датафрейма столбцы `education` и `family_status`, оставив только их идентификаторы: `education_id` и `family_status_id`. Новые датафреймы — это те самые «словари», к которым мы сможем обращаться по идентификатору.

In [61]:
# удаляем education и family_status из df
df=df.drop(['education', 'family_status'], axis=1)
df.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,flag_change_income
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья,NaN
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,NaN
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья,NaN
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование,NaN
4,0,2249,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,NaN


### Шаг 2.6. Категоризация дохода.

Для исследования создадим категории дохода заемщиков.
На основании диапазонов, указанных ниже, создаем столбец `total_income_category` с категориями:
* 0–30000 — 'E';
* 30001–50000 — 'D';
* 50001–200000 — 'C';
* 200001–1000000 — 'B';
* 100001 и выше — 'A'.


In [62]:
# создаем функцию, которая распределяет клиентов по категории в зависимости от их дохода

def category_total_income(row):
    """
    Функция возвращает категорию дохода клиента:
       0–30000 — 'E';
       30001–50000 — 'D';
       50001–200000 — 'C';
       200001–1000000 — 'B';
       100001 и выше — 'A'.
    
    """
     
    if 0<=row<=30000:
        return 'E'
    elif row<=50000:
        return 'D'
    elif row<=200000:
        return 'C'
    elif row<=1000000:
        return 'B'
    return 'A'

# создаем столбец total_income_category в df
df['total_income_category']=df['total_income'].apply(category_total_income)

# смотрим первые 10 строчек df
df.head(10)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,flag_change_income,total_income_category
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья,NaN,B
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,NaN,C
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья,NaN,C
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование,NaN,B
4,0,2249,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,NaN,C
5,0,926,27,0,1,M,компаньон,0,255763,покупка жилья,NaN,B
6,0,2879,43,0,0,F,компаньон,0,240525,операции с жильем,NaN,B
7,0,152,50,1,0,M,сотрудник,0,135823,образование,NaN,C
8,2,6929,35,0,1,F,сотрудник,0,95856,на проведение свадьбы,NaN,C
9,0,2188,41,1,0,M,сотрудник,0,144425,покупка жилья для семьи,NaN,C


Котегоризация по уровню дохода прошла успешно!

### Шаг 2.7. Категоризация целей кредита.

В шаге 2.2 видим в колонке `purpose` много уникальных значений целей кредита. Посмотрим на них еще раз.


In [63]:
# выводим уникальные значения purpose
df['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Значения для удобства исследования можно обобщить в категории:

* операции с автомобилем
* операции с недвижимостью
* проведение свадьбы
* получение образования

Создадим новый столбец `purpose_category` в df и присвоим клиентам категории на основе данных из `purpose`

In [64]:
# автомобил свадьб образован
# создаем функцию, которая распределяет клиентов по категории в зависимости от цели кредита

def category_purpose(row):
    """
    Функция возвращает категорию цели кредита:
       операции с автомобилем
       операции с недвижимостью
       проведение свадьбы
       получение образования
    
    """
     
    if 'автомобил' in row:
        return 'операции с автомобилем'
    elif 'свадьб' in row:
        return 'проведение свадьбы'
    elif 'образован' in row:
        return 'получение образования'
    return 'операции с недвижимостью'

# создаем столбец purpose_category в df
df['purpose_category']=df['purpose'].apply(category_purpose)

# смотрим первые 10 строчек df
df.head(10)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,flag_change_income,total_income_category,purpose_category
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья,NaN,B,операции с недвижимостью
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,NaN,C,операции с автомобилем
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья,NaN,C,операции с недвижимостью
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование,NaN,B,получение образования
4,0,2249,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,NaN,C,проведение свадьбы
5,0,926,27,0,1,M,компаньон,0,255763,покупка жилья,NaN,B,операции с недвижимостью
6,0,2879,43,0,0,F,компаньон,0,240525,операции с жильем,NaN,B,операции с недвижимостью
7,0,152,50,1,0,M,сотрудник,0,135823,образование,NaN,C,получение образования
8,2,6929,35,0,1,F,сотрудник,0,95856,на проведение свадьбы,NaN,C,проведение свадьбы
9,0,2188,41,1,0,M,сотрудник,0,144425,покупка жилья для семьи,NaN,C,операции с недвижимостью


Категоризация по цели кредитования прошла успешно.

Данные готовы к исследованию.

**Выводы**

Предобработка обнаружила проблемы в данных:

- пропуски в двух столбцах,
- аномальные значения,
- дубликаты.

Что было сделано:
- заполнены 10% пропусков в колонке `total_income` медианными значениями с группировкой по типу занятости и уровню образования.
- аномалии в `days_employed` и `dob_years` исправлены в учебных целях
- аномалии в `children` - исправлены, в `gender` - удалены
- удалены неявные дубликаты в `education`
- значения столбцов `education` и `family_status` перенесены в словари
- выделены категории `total_income_category` по уровню дохода и `purpose_category` по цели кредитования

Теперь можно перейти к вопросам исследования. 

## Шаг 3. Ответы на вопросы. <a id='conclusion'></a>

### Вопрос 1:

Есть ли зависимость между количеством детей и возвратом кредита в срок?

Обычно людям, обременненным детьми не охотно предоставляют кредиты, исходя из большей обремененности родителей. Посмотрим, справедливы ли эти опасения кредиторов?

Для ответа на этот вопрос посмотрим кол-во задолженностей из колонки `debt` относительно кол-ву детей в семье из колонки `children`

In [65]:
# Подсчёт количества клиентов с задолженностью по кол-ву детей в семье
df.groupby(by='children')['debt'].sum()


children
0    1063
1     445
2     202
3      27
4       4
5       0
Name: debt, dtype: int64

Неожиданный поворот: чем больше детей, тем больше ответсвенности перед банком, если смотреть на сухие данные по должникам. Бездетных должников больше, чем должников-родителей. Но бездетных клиентов тоже больше. Посмотрим на процентное соотношение должников из каждой категории к общему числу.

In [79]:
# создаем функцию для отображения сводной таблицы с процентным соотношением доли.
def pivot_table_ratio (column):
    
    # построим сводную таблицу и посчитаем долю задолженности по отношению к общему числу в группе
    df_pivot=df.pivot_table(index=column, columns='debt', values='gender', aggfunc='count')

    # считаем и округляем процентное соотношение
    df_pivot['ratio']= df.groupby(by=column).agg({'debt': ['mean']})*100
    df_pivot=df_pivot.round(1)

    return df_pivot

In [80]:
# вызываем функцию для колонки children
pivot_table_ratio('children')

debt,0,1,ratio
children,,,
0,13027.0,1063.0,7.5
1,4410.0,445.0,9.2
2,1926.0,202.0,9.5
3,303.0,27.0,8.2
4,37.0,4.0,9.8
5,9.0,NaN,0.0


И все-таки без детей кредиты выплачиваются лучше всего 7,5% должников из этой группы.

Многодетные семьи с 3-я детьми идут на втором месте - 8,2%

в 9,2% случаев задолженности у семей с 1 ребенком.

Семьям с 2-мя и 4-мя детьми сложнее всего отдавать кредиты -9,5 - 9,8% должников.

Только наличие 5 детей полностью гарантирует своевременную оплату долга. Но кол-во таких семей в данных всего 9, что не очень показательно.

#### Вывод 1:

Клиенты без детей лучше выплачивают кредиты на 0,7-2,3% лучше, чем с детьми. Прямой зависимости кол-ва задолженностей от кол-ва детей в семьях с детьми нет.

### Вопрос 2:

Есть ли зависимость между семейным положением и возвратом кредита в срок?

Ячейке общества из двух трудоспособных людей проще нести бремя долговой отвественности, чем одинокому человеку?

Сгруппируем данные из `debt` по семейному положению и отобразим значения статусов из `family_dict`

In [81]:
# отображаем словарь family_dict
family_dict

,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,не женат / не замужем,4


In [82]:
# Подсчёт количества клиентов с задолженностью по семейному положению
df.groupby(by='family_status_id')['debt'].sum()

family_status_id
0    931
1    388
2     63
3     85
4    274
Name: debt, dtype: int64

"У двух кредиторов кредит без присмотра" можно сказать, глядя на сухую выборку по должникам.
Лидеры по задолженностям с хорошим отрывом пары в официальном браке, за ними идет гражданский брак и клиенты не в браке. Зато овдовевшие и в разводе реже попадают в список должников.

Но посмотрим на долю должников по каждой группе.

In [83]:
pivot_table_ratio('family_status_id')

debt,0,1,ratio
family_status_id,,,
0,11408,931,7.5
1,3762,388,9.3
2,896,63,6.6
3,1110,85,7.1
4,2536,274,9.8


In [84]:
# вызываем функцию для колонки family_status_id добавим в сводную таблицу расшифровку family_status_id из family_dict
df_pivot = pivot_table_ratio('family_status_id').merge(family_dict, on = 'family_status_id', how='left')
df_pivot

,family_status_id,0,1,ratio,family_status
0,0,11408,931,7.5,женат / замужем
1,1,3762,388,9.3,гражданский брак
2,2,896,63,6.6,вдовец / вдова
3,3,1110,85,7.1,в разводе
4,4,2536,274,9.8,не женат / не замужем


Опять первое впечатление обманчиво. 

По доле в общем числе группы лидируют люди без семьи - 9,8% должников среди них; и клиенты в гражданском браке - 9,3%.

В официальном браке и после него с отвественностью лучше: 7,5% и 7,1% должников.

А овдовевшие люди реже всего задерживают выплаты - 6,6%

#### Вывод 2:

Клиенты без официально-зарегистрированных отношений хуже возвращают кредиты, чем клиенты в браке или после него на 0,5-3,2%

### Вопрос 3:

Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Чем выше доход, тем легче платить по кредиту?

Сразу посмотрим на долю должников по каждой группе дохода.

In [85]:
# вызываем функцию для колонки total_income_category
pivot_table_ratio('total_income_category')

debt,0,1,ratio
total_income_category,,,
A,23,2,8.0
B,4863,364,7.0
C,14477,1352,8.5
D,329,21,6.0
E,20,2,9.1


Категории дохода от A до Е, где А - самые обеспеченные клиенты, а Е - самый низкий доход.
По распределению дохода прямой пропорции к возврату долгов не наблюдается. Ожидаемо хуже всего с выплатами у клиентов с самым низким доходом (до 30тр) - 9,1%, но самый низкий процент должников 6% у следующей категории 30-50тр. в 7% случаях задерживают выплаты клиенты с высоким доходом 200тр-1млн. А самые богатые (>1млн) в 8%. Средний доход (50-200тр) возвращают долги почти также как и люди с самым низким доходом - 8,5%

Также надо не забывать, что 10% пропусков в колонке с доходами были заменены медианными значениями с группировкой по типу занятости и уровню образования.

Посмотрим на распределение клиентов с изначально пропущенным доходом и какая доля должников среди них.

In [86]:
# вызываем функцию для колонки total_income_category с сортировкой по #flag_change_income
pivot_table_ratio([df['flag_change_income']==1, 'total_income_category'])

debt                                          0     1  ratio
flag_change_income total_income_category                    
False              A                         23     2    8.0
                   B                       4684   356    7.1
                   C                      12723  1190    8.6
                   D                        329    21    6.0
                   E                         20     2    9.1
True               B                        179     8    4.3
                   C                       1754   162    8.5

Изменения в заполнении коснулись только категории В и С. До заполнения пропусков значения были 7,1% и 8,6%, а после 7,0% и 8,5%. Из этого можно сделать вывод, что заполнение пропусков не сильно исказило значения в процентном соотношении.

#### Вывод 3:

По распределению дохода прямой пропорции к возврату долгов не наблюдается. Хуже всего с возвратом долгов у самых бедных и клиентов со средним доходом. Лучше всего платят по кредитам клиенты с небольшим доходом. А самые богатые клиенты позволяют себе пропускать платежи чаще. Заполнение пропущенных значений в доходе медианой не сильно исказило данные. 

### Вопрос 4:

Как разные цели кредита влияют на его возврат в срок?

Никаких предположений пока нет. Посмотрим на долю должников по каждой группе дохода.

In [87]:
# вызываем функцию для колонки purpose_category
pivot_table_ratio('purpose_category')

debt,0,1,ratio
purpose_category,,,
операции с автомобилем,3903,403,9.4
операции с недвижимостью,10028,782,7.2
получение образования,3643,370,9.2
проведение свадьбы,2138,186,8.0


По операциям с недвижимостью самый маленький процент просрочек - 7,2%

На свадьбу кредит берут реже, а процент возврата на 2-м месте - 8,0%

Образование и автомобили - цели, по которым задерживают чаще всего - 9,2%, 9,4%

#### Вывод 4:
Разброс по долям 2,2%. Не самый маленький. Наблюдается зависимость цели кредита от задержек в выплате. Чаще всего задерживают по целям Образование и Автомобили, а цели Свадьба и Недвижимость выплачивают лучше.

## Общий вывод:

Мы провели исследование с целью ответить на вопросы по возвращаемости кредитов в зависимости от семейного положения, наличия и кол-ва детей, целей кредита и дохода заемщика.

Можно резюмировать:

Меньше задолженностей у овдовевших и разведенных людей, без детей, с небольшим доходом 30-50тр, по целям Свадьба и Недвижимость.

Хуже возвращают кредиты клиенты без партнера или в гражданском браке, с 4-мя детьми, с низким доходом до 30тр и по целям Образование и Автомобиль.

Внесенные изменения в данные в статье доходы не сильно исказили результаты. А по сильно измененным колонкам Стаж и Возраст исследование не проводилось. Остальные изменения были не значительны.